## Input Parameters

In [1]:
# This would show all NLCD classes for MGRS grid zone 15S
point_nlcd_keep_list = [31, 41, 42, 43, 52, 71, 81, 82, 90, 95]
point_mgrs_keep_list = ['15S']

# The minimum number of months in the target year
min_month_count = 3

# Overwrite the summary stats file everytime this block is run
output_txt = 'summary_stats_mgrs15s.txt'
output_f = open(output_txt, 'w')
output_f.write('MGRS: '+ ', '.join(point_mgrs_keep_list) + '\n')
output_f.write('NLCD: '+ ', '.join(map(str, point_nlcd_keep_list)) + '\n\n')
output_f.close()

## Other Parameters

In [2]:
data_folder = 'data'
points_folder = 'points'

points_csv = 'gap_fill_test_points.csv'

# Exclude 2016 from the statistics since there is not a full prior year to interpolate from
# Including 2024 even though 2025 is not complete
stats_years = list(range(2017, 2025))

# Points were only built for NLCD 2024
nlcd_years = [2024]

# Include all MGRS grid zones except 12R and 16U since they are too small
mgrs_zones = [
    '10S', '10T', '10U', '11S', '11T', '11U', '12S', '12T', '12U', 
    '13R', '13S', '13T', '13U', '14R', '14S', '14T', '14U', '15R', '15S', '15T', '15U', 
    '16R', '16S', '16T', '17R', '17S', '17T', '18S', '18T', '19T'
    # '12R', '16U'
]

months = list(range(1, 13))

# TODO: Add support for setting a minimum number of months in the year
#   and minimum number of months in the growing season
# min_month_count = 6
# min_gs_month_count = 3


## Python Imports

In [3]:
from datetime import datetime
import math
import os
import pprint
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.metrics
import sklearn.linear_model

from whittaker_eilers import WhittakerSmoother

## Read the point CSV files

In [4]:
# Building a single points dataframe and CSV from the MGRS grid zone points CSV files
overwrite_flag = False

# Read the separate points CSV files into a single dataframe
points_df_list = [
    pd.read_csv(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'), index_col=None, header=0)
    for nlcd_year in nlcd_years
    for mgrs_zone in mgrs_zones
    if os.path.isfile(os.path.join(points_folder, f'points_{mgrs_zone}_{nlcd_year}.csv'))
]
points_df = pd.concat(points_df_list, axis=0, ignore_index=True)
print(f'Points: {len(points_df.index)}')

# The mgrs_zone value will eventually be added to the csv files
points_df['mgrs_zone'] = points_df['mgrs_tile'].str.slice(0, 3)

# Add a unique index to the points dataframe
points_df['index_group'] = points_df.groupby(['mgrs_tile', 'nlcd']).cumcount()
points_df['point_id'] = (
    points_df["mgrs_tile"].str.upper() + '_' +
    'nlcd' + points_df["nlcd"].astype(str).str.zfill(2) + '_' +
    points_df["index_group"].astype(str).str.zfill(2)
)
del points_df['index_group']

# Round the lat and lon to 8 decimal places (probably should be 6)
points_df['latitude'] = round(points_df['latitude'], 8)
points_df['longitude'] = round(points_df['longitude'], 8)

# # Write to CSV
# if not os.path.isfile(points_csv) or overwrite_flag:
#     print('Writing points csv')
#     points_df.to_csv(points_csv, index=False)


Points: 17084


## Read the data CSV files

This block may take a little while to execute

In [5]:
# Read the CSV files into separate dataframes for each point
print('Reading mgrs data csv files')
data_df_dict = {}
for mgrs_zone in mgrs_zones:
    # print(mgrs_zone)
    if not os.path.isfile(os.path.join(data_folder, f'data_{mgrs_zone}.csv')):
        continue
        
    mgrs_df = pd.read_csv(os.path.join(data_folder, f'data_{mgrs_zone}.csv'), index_col=None, header=0)

    # Set MGRS value to upper case 
    # (at some point change this in all the data CSV files)
    mgrs_df['mgrs_tile'] = mgrs_df['mgrs_tile'].str.upper()
    mgrs_df['mgrs_zone'] = mgrs_df['mgrs_zone'].str.upper()
    
    # Compute the ET fraction
    mgrs_df['etf'] = mgrs_df['et'] / mgrs_df['eto']
    
    # Get the month for computing climos
    mgrs_df['date'] = pd.to_datetime(mgrs_df['date'])
    mgrs_df['year'] = mgrs_df['date'].dt.year
    mgrs_df['month'] = mgrs_df['date'].dt.month
    
    # Confirm that specific NLCD categories are not included
    # TODO: This probably isn't needed and switch to a check instead of masking
    for nlcd_skip in [11, 12, 21, 22, 23]:
        mgrs_df = mgrs_df[mgrs_df['nlcd'] != nlcd_skip]

    # Save dataframe for each point
    for point_id in mgrs_df['point_id'].unique():
        site_df = mgrs_df.loc[mgrs_df['point_id']==point_id].copy()
        site_df.set_index('date', drop=True, inplace=True)
        site_df.sort_index(inplace=True)
        data_df_dict[point_id] = site_df
        del site_df
    del mgrs_df

print('\nDone')

Reading mgrs data csv files

Done


## Compute the ETf climos

In [6]:
# Compute the maximum ETf per site
# Assuming it is okay to make this for the full period of record
print('\nComputing maximum ETf')
etf_max_dict = {
    point_id: data_df_dict[point_id].agg(etf=('etf', 'max'))['etf'].to_dict()['etf']
    for point_id in data_df_dict.keys()
}

# Compute climos for each site
# Only keep the climo value if there are at least "n" years of data
print('\nComputing monthly climatologies')
month_climo_count_min = 2
month_climo_dict = {}
for point_id in data_df_dict.keys():
    month_climo = data_df_dict[point_id].groupby(['month']).agg(
        etf=('etf', 'mean'), 
        etf_median=('etf', 'median'), 
        count=('etf', 'count'), 
        et=('et', 'mean'), 
        eto=('eto', 'mean'),
    )
    month_climo_count_mask = month_climo['count'] < month_climo_count_min
    month_climo.loc[month_climo_count_mask, ['etf', 'etf_median', 'et']] = np.nan
    month_climo_dict[point_id] = month_climo
    del month_climo, month_climo_count_mask

# # Compute climos for each target year that have the target year values excluded
# # CGM - This might be worth testing more but doesn't seem worth the time to generate
# print('\nComputing monthly climatologies for target years')
# month_climo_dict = {}
# for point_id in data_df_dict.keys():
#     if point_id not in month_climo_dict.keys():
#         month_climo_dict[point_id] = {}
#     for year in stats_years:
#         month_climo_dict[point_id][year] = (
#             data_df_dict[point_id][data_df_dict[point_id].year != year]
#             .groupby(['month'])
#             .agg(
#                 etf=('etf', 'mean'), 
#                 etf_median=('etf', 'median'), 
#                 count=('etf', 'count'), 
#                 et=('et', 'mean'), 
#                 eto=('eto', 'mean'),
#             )
#         )

# TODO: Join the climo values to the data dictionaries
#   It might be faster to join the climo data here instead of looking up in the function

print('\nDone')


Computing maximum ETf

Computing monthly climatologies

Done


## Functions for computing filled values and summary statistics

In [7]:
def generate_windows(
        point_id_list, 
        months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 
        years=[2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024], 
        exclude_months_without_climo=True
):
    """Generate Window Dataframes"""
    for i, point_id in enumerate(point_id_list):
        # Assume the data df dictionary exists in the global scope
        site_df = data_df_dict[point_id]
        
        for year in years:
            # Pull a three year window for each target year so that there are images to interpolate and fill from
            window_df = site_df[(site_df.index.year >= (year-1)) & (site_df.index.year <= (year+1))].copy()

            # If excluding months without climos, set them to NaN here
            # TODO: Test out adding the climo values to the window_df here 
            #   instead of in the compute_filled_values() function below
            if exclude_months_without_climo:
                merge_df = pd.merge(window_df[['month']], month_climo_dict[point_id]['etf'], how="left", on="month")
                climo_nan_mask = merge_df['etf'].isna().values
                window_df.loc[climo_nan_mask, ['etf', 'et', 'count']] = np.nan

            year_mask = window_df.index.year==year
            year_month_mask = year_mask & window_df.index.month.isin(months)
            
            if window_df.loc[year_mask, 'etf'].count() < min_month_count:
                # Check if there are enough months in the target year
                # print(f'{point_id} - {i} - {year} - not enough unmasked months, skipping')
                continue
            elif window_df.loc[year_month_mask, 'etf'].isna().all():
                # Check that there are target months with data in the 
                # print(f'{point_id} - {i} - {year} - no unmasked months in year/months, skipping')
                continue
            elif (window_df.loc[(window_df.index.year==year-1), 'etf'].isna().all() or 
                  window_df.loc[(window_df.index.year==year+1), 'etf'].isna().all()):
                # Check if there is data in the prev/next year to interpolate from
                # print(f'{point_id} - {i} - {year} - no unmasked months in next/prev year, skipping')
                continue

            yield point_id, year, window_df, year_month_mask


comparison_cols = [
    'interpolate', 'climo_mean', 'climo_median',
    'conor',
    'interp_clim_a', 'interp_clim_b', 'interp_clim_c',
    'whit_a_0p50', 'whit_a_0p20', 'whit_a_0p10', 'whit_a_0p05', 'whit_a_0p01', 
]


def compute_filled_values(window_df, tgt_indices, point_id):
    """"""
    # Get a copy of the target value before clearing
    original_etf = window_df.loc[tgt_indices, 'etf'].values
        
    # Set the target row values to NaN
    window_df.loc[tgt_indices, ('etf', 'et', 'count')] = np.nan
    
    # Setup the Whittaker Smoothing for the full dataframe outside of the index loop
    # The smoothing function needs all nans filled with a value
    # The fill value is not important as long as the weight value is set to 0
    window_df['temp'] = window_df['etf'].copy()
    window_df.loc[np.isnan(window_df['temp']), 'temp'] = -1
    etf = window_df['temp'].values

    # TODO: Make sure weights are set to 0 for all temp==-1 rows
    #   This might be happening already with the .fillna(0) call but double check
    #   Right now the code is assuming count is NaN if etf is NaN
    
    # Default weights with 1 for data and 0 for missing values
    weight_a = window_df['count'].clip(lower=1, upper=1).fillna(0)
    if not any(weight_a):
        print(f'{point_id} - {i} - {year} - all weights 0, skipping')
        return []
    # CGM - I tested out building the smoother once and then updating lambda,
    #   but it didn't seem any faster
    whit_a_0p50 = WhittakerSmoother(lmbda=0.5, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p20 = WhittakerSmoother(lmbda=0.2, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p10 = WhittakerSmoother(lmbda=0.1, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p05 = WhittakerSmoother(lmbda=0.05, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)
    whit_a_0p01 = WhittakerSmoother(lmbda=0.01, order=2, data_length=len(weight_a), weights=weight_a).smooth(etf)

    # CGM - I was testing out trying different weights but it didn't seem to change the values at all
    # # Compute weights based on the the scene count value
    # # Set count 0 images to a weight of 0
    # weight = window_df['count'].clip(lower=0, upper=1).fillna(0)

    # # Compute weights based on the the scene count value
    # # Set counts of 0 to a weight of 0.5 and all other to 1    
    # weight = window_df['count'].add(1).clip(upper=2).divide(2).fillna(0)

    # # Compute weights based on the scene count value
    # # Set count weights as: 0 -> 0, 1 -> 0.5, 2+ -> 1
    # weight = window_df['count'].fillna(0).clip(upper=2).divide(2)
    
    # Process each target index separately
    values = []
    for i, (tgt_index, tgt_i) in enumerate(zip(tgt_indices, window_df.index.get_indexer(tgt_indices))):

        interp_value = window_df['etf'].interpolate(method='linear').loc[tgt_index]

        # Climos for all years
        climo_mean = month_climo_dict[point_id].loc[tgt_index.month, 'etf']
        climo_count = month_climo_dict[point_id].loc[tgt_index.month, 'count']
        climo_median = month_climo_dict[point_id].loc[tgt_index.month, 'etf_median']
        # # Climos with the target year excluded (not sure if this matters)
        # climo_mean = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf']
        # climo_count = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'count']
        # climo_median = month_climo_dict[point_id][tgt_index.year].loc[tgt_index.month, 'etf_median']

        # Compute various combinations of averaging the climo and interpolate values
        # Simple mean
        interp_clim_a = (climo_mean + interp_value) / 2
        # Simple mean with the median climo
        interp_clim_c = (climo_median + interp_value) / 2
        # Weight the climo based on the number of months in the climo?
        climo_months = 10
        interp_clim_b = (climo_mean * climo_count + interp_value * climo_months) / (climo_count + climo_months)

        # Conor's Approach
        # There is probably an easier way, but splitting the dataframe at the target index seemed to work pretty well
        window_prev_df = window_df.iloc[:tgt_i]
        window_next_df = window_df.iloc[tgt_i+1:]
        prev_index = window_prev_df['etf'].last_valid_index()
        next_index = window_next_df['etf'].first_valid_index()
        prev_i = window_df.index.get_loc(prev_index)
        next_i = window_df.index.get_loc(next_index)
        w_prev = 0.5 * math.exp(1 - (tgt_i - prev_i))
        w_next = 0.5 * math.exp(1 - (next_i - tgt_i))
        value_prev = window_df['etf'].iloc[prev_i]
        value_next = window_df['etf'].iloc[next_i]
        climo_prev = month_climo_dict[point_id].loc[prev_index.month, 'etf']
        climo_next = month_climo_dict[point_id].loc[next_index.month, 'etf']
        conor = w_prev * (value_prev - climo_prev) + w_next * (value_next - climo_next) + climo_mean

        values.append({
            'index': tgt_index,
            'point_id': point_id,
            'mgrs': point_id.split('_')[0],
            'nlcd': int(point_id.split('_')[1][4:6]),
            'original': original_etf[i],
            # Filled values
            'interpolate': interp_value,
            'climo_mean': climo_mean,
            'climo_median': climo_median,
            'conor': conor,
            'interp_clim_a': interp_clim_a,
            'interp_clim_b': interp_clim_b,
            'interp_clim_c': interp_clim_c,
            'whit_a_0p50': min(max(whit_a_0p50[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p20': min(max(whit_a_0p20[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p10': min(max(whit_a_0p10[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p05': min(max(whit_a_0p05[tgt_i], 0), etf_max_dict[point_id]),
            'whit_a_0p01': min(max(whit_a_0p01[tgt_i], 0), etf_max_dict[point_id]),
            
        })

    return values


def comparison_stats(df, x_col='original', y_cols=[], title='', print_flag=True, write_flag=True):
    """"""
    output = [title]

    # TODO: Build the format strings based on the number of parameters instead of hardcoding
    output.append('  {:>16s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s} {:>8s}'.format(
        'method', 'rmse', 'mae', 'mbe', 'm', 'b', 'r2', 'n'
    ))
    for y_col in y_cols:
        # Remove any NaN rows before computing statistics
        stat_df = df[df[y_col].notna()]
        model = sklearn.linear_model.LinearRegression()
        model.fit(stat_df[[x_col]], stat_df[y_col])

        output.append('  {:>16s} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8d}'.format(
            y_col,
            sklearn.metrics.root_mean_squared_error(stat_df[x_col], stat_df[y_col]),
            sklearn.metrics.mean_absolute_error(stat_df[x_col], stat_df[y_col]),
            np.mean(stat_df[y_col] - stat_df[x_col]),
            # np.mean(stat_df[x_col] - stat_df[y_col]),
            # sklearn.metrics.r2_score(stat_df[x_col], stat_df[y_col]),
            model.coef_[0],
            model.intercept_, 
            model.score(stat_df[[x_col]], stat_df[y_col]),
            # This count doesn't seem to change even when there are NaN values in the dataframe
            stat_df[y_col].count(),
        ))

    if print_flag:
        print('\n'.join(output))
    if write_flag:
        with open(output_txt, 'a') as output_f:
            output_f.write('\n'.join(output+ ['\n']))


## Filter to the target MGRS and NLCD classes

In [8]:
# Filter the points list to the target NLCD classes and MGRS grid zones
point_id_list = list(data_df_dict.keys())
if point_nlcd_keep_list:
    point_id_list = [p for p in point_id_list if int(p.split('_')[1][4:6]) in point_nlcd_keep_list]
if point_mgrs_keep_list:
    point_id_list = [p for p in point_id_list if p.split('_')[0][0:3] in point_mgrs_keep_list]

print(f'Points: {len(point_id_list)}')

Points: 960


## Randomly drop one datapoint in each year

In [9]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop one datapoint in each year',
    print_flag=True, write_flag=True
)

Randomly drop one datapoint in each year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1139   0.0878  -0.0001   0.7867   0.1646   0.8231     7680
        climo_mean   0.1208   0.0904   0.0023   0.7988   0.1576   0.7992     7680
      climo_median   0.1246   0.0866   0.0070   0.8236   0.1432   0.7884     7680
             conor   0.0979   0.0750  -0.0018   0.8986   0.0765   0.8691     7680
     interp_clim_a   0.1024   0.0791   0.0011   0.7927   0.1611   0.8612     7680
     interp_clim_b   0.1024   0.0791   0.0012   0.7925   0.1614   0.8611     7680
     interp_clim_c   0.1028   0.0778   0.0035   0.8051   0.1539   0.8581     7680
       whit_a_0p50   0.1145   0.0884  -0.0006   0.8199   0.1384   0.8197     7680
       whit_a_0p20   0.1128   0.0864  -0.0008   0.8497   0.1153   0.8257     7680
       whit_a_0p10   0.1127   0.0861  -0.0009   0.8636   0.1045   0.8267     7680
       whit_a_0p05   0.1133   0.0864  -0.0009   0.8724   

## Randomly drop a single datapoint from the "growing" season (Apr-Sept)

In [10]:
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
        
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "growing" season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "growing" season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1007   0.0774  -0.0232   0.7824   0.1818   0.7730     7680
        climo_mean   0.1170   0.0853   0.0059   0.6720   0.3148   0.6773     7680
      climo_median   0.1217   0.0823   0.0165   0.6807   0.3173   0.6570     7680
             conor   0.0860   0.0651   0.0009   0.8593   0.1334   0.8266     7680
     interp_clim_a   0.0939   0.0718  -0.0087   0.7272   0.2483   0.7998     7680
     interp_clim_b   0.0939   0.0718  -0.0088   0.7274   0.2480   0.7999     7680
     interp_clim_c   0.0945   0.0703  -0.0034   0.7315   0.2496   0.7939     7680
       whit_a_0p50   0.1029   0.0787  -0.0118   0.7957   0.1806   0.7552     7680
       whit_a_0p20   0.0997   0.0761  -0.0063   0.8211   0.1623   0.7693     7680
       whit_a_0p10   0.0986   0.0753  -0.0041   0.8338   0.1525   0.7748     7680
       whit_a_0p05   0.0984 

## Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)

In [11]:
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint from the "non-growing" season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1276   0.0998   0.0283   0.6946   0.2119   0.6581     7680
        climo_mean   0.1301   0.0994   0.0040   0.6225   0.2309   0.6241     7680
      climo_median   0.1340   0.0960   0.0026   0.6442   0.2164   0.6041     7680
             conor   0.1105   0.0862   0.0011   0.7907   0.1269   0.7331     7680
     interp_clim_a   0.1138   0.0891   0.0161   0.6585   0.2214   0.7238     7680
     interp_clim_b   0.1139   0.0892   0.0164   0.6588   0.2215   0.7232     7680
     interp_clim_c   0.1147   0.0882   0.0154   0.6694   0.2141   0.7162     7680
       whit_a_0p50   0.1263   0.0986   0.0165   0.7051   0.1937   0.6554     7680
       whit_a_0p20   0.1258   0.0981   0.0107   0.7331   0.1711   0.6590     7680
       whit_a_0p10   0.1267   0.0986   0.0083   0.7474   0.1601   0.6573     7680
       whit_a_0p05   0.12

## Randomly drop a datapoint from the winter (Dec-Feb)

In [12]:
months = [12, 1, 2]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        
    tgt_mask = year_month_mask & window_df['etf'].notna()
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))

output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a datapoint from the winter (Dec-Feb)',
    print_flag=True, write_flag=True
)

Randomly drop a datapoint from the winter (Dec-Feb)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1331   0.1041   0.0352   0.6512   0.2193   0.5626     7675
        climo_mean   0.1322   0.1010   0.0051   0.5243   0.2561   0.5222     7675
      climo_median   0.1357   0.0974   0.0019   0.5431   0.2429   0.4995     7675
             conor   0.1159   0.0898   0.0026   0.7106   0.1553   0.6402     7675
     interp_clim_a   0.1184   0.0920   0.0202   0.5877   0.2377   0.6298     7675
     interp_clim_b   0.1185   0.0921   0.0205   0.5892   0.2373   0.6298     7675
     interp_clim_c   0.1193   0.0912   0.0186   0.5972   0.2311   0.6207     7675
       whit_a_0p50   0.1308   0.1016   0.0158   0.6574   0.1965   0.5567     7675
       whit_a_0p20   0.1310   0.1015   0.0079   0.6778   0.1779   0.5578     7675
       whit_a_0p10   0.1322   0.1024   0.0054   0.6887   0.1697   0.5546     7675
       whit_a_0p05   0.1337   0.1034   0.0044 

## Randomly drop a single datapoint that is next to an existing missing data point 

In [13]:
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # Skip the year if there are no NaN values
    if not window_df.loc[year_month_mask, 'etf'].isna().any():
        # print(f'{point_id} - {i} - {year} - no missing data points, skipping')
        continue
    
    # For the target year, pick a random month that is next to a missing/masked month but has data
    nan_mask = window_df['etf'].isna()
    tgt_mask = (
        (nan_mask | nan_mask.shift(1) | nan_mask.shift(-1))
        & window_df['etf'].notna() & year_month_mask
    )
    if not tgt_mask.any():
        print(f'{point_id} - {i} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index
    output_list.extend(compute_filled_values(window_df, tgt_indices, point_id))
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a single datapoint that is next to an existing missing data point',
    print_flag=True, write_flag=True
)

Randomly drop a single datapoint that is next to an existing missing data point
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1667   0.1314   0.0320   0.5952   0.3187   0.6361     1469
        climo_mean   0.1476   0.1118  -0.0189   0.6889   0.2014   0.7069     1469
      climo_median   0.1532   0.1095  -0.0170   0.7048   0.1921   0.6834     1469
             conor   0.1316   0.1024  -0.0065   0.7455   0.1737   0.7638     1469
     interp_clim_a   0.1390   0.1092   0.0065   0.6420   0.2600   0.7522     1469
     interp_clim_b   0.1395   0.1097   0.0075   0.6401   0.2624   0.7504     1469
     interp_clim_c   0.1400   0.1087   0.0075   0.6500   0.2554   0.7441     1469
       whit_a_0p50   0.1605   0.1255   0.0145   0.6693   0.2487   0.6509     1469
       whit_a_0p20   0.1601   0.1247   0.0104   0.7060   0.2186   0.6547     1469
       whit_a_0p10   0.1615   0.1256   0.0091   0.7216   0.2063   0.6514     1469
       whit_a_0p05

## Randomly drop a two month gap during the year

But only check the filled value in one month of the gap

In [14]:
dropped_months = 2
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))

    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1420   0.1110   0.0038   0.6794   0.2505   0.7274     7680
        climo_mean   0.1272   0.0944   0.0055   0.7826   0.1728   0.7791     7680
      climo_median   0.1320   0.0913   0.0101   0.8061   0.1594   0.7653     7680
             conor   0.1081   0.0817   0.0030   0.8439   0.1231   0.8404     7680
     interp_clim_a   0.1173   0.0907   0.0046   0.7310   0.2117   0.8221     7680
     interp_clim_b   0.1175   0.0909   0.0048   0.7302   0.2125   0.8216     7680
     interp_clim_c   0.1178   0.0895   0.0069   0.7427   0.2050   0.8176     7680
       whit_a_0p50   0.1383   0.1064   0.0034   0.7583   0.1894   0.7388     7680
       whit_a_0p20   0.1383   0.1062   0.0031   0.7921   0.1632   0.7417     7680
       whit_a_0p10   0.1395   0.1071   0.0030   0.8075   0.1512   0.7399     7680
       whit_a_0p05   0.1411   0.1082   0.0029   0.81

## Randomly drop a two month gap during the growing season (Apr-Sept)

In [15]:
dropped_months = 2
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1280   0.0998  -0.0511   0.7232   0.2091   0.6799     7680
        climo_mean   0.1200   0.0869   0.0049   0.6618   0.3228   0.6634     7680
      climo_median   0.1254   0.0841   0.0152   0.6685   0.3268   0.6386     7680
             conor   0.0966   0.0728   0.0018   0.7754   0.2129   0.7815     7680
     interp_clim_a   0.1061   0.0819  -0.0231   0.6925   0.2660   0.7535     7680
     interp_clim_b   0.1062   0.0819  -0.0232   0.6926   0.2657   0.7534     7680
     interp_clim_c   0.1063   0.0803  -0.0179   0.6958   0.2680   0.7465     7680
       whit_a_0p50   0.1256   0.0957  -0.0272   0.7620   0.1965   0.6626     7680
       whit_a_0p20   0.1244   0.0947  -0.0200   0.7892   0.1782   0.6686     7680
       whit_a_0p10   0.1247   0.0951  -0.0170   0.8021   0.1690   0.6690     7680
       whit_a_0p05   0.1255   0

## Randomly drop a two month gap during the non-growing season (Oct-Mar)

In [16]:
dropped_months = 2
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a two month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a two month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1501   0.1178   0.0554   0.6511   0.2650   0.5875     7680
        climo_mean   0.1302   0.0989   0.0031   0.6308   0.2249   0.6351     7680
      climo_median   0.1345   0.0954   0.0022   0.6523   0.2112   0.6130     7680
             conor   0.1134   0.0871   0.0013   0.7285   0.1644   0.7232     7680
     interp_clim_a   0.1227   0.0958   0.0293   0.6409   0.2450   0.6986     7680
     interp_clim_b   0.1231   0.0961   0.0298   0.6408   0.2456   0.6974     7680
     interp_clim_c   0.1235   0.0949   0.0288   0.6517   0.2381   0.6919     7680
       whit_a_0p50   0.1446   0.1130   0.0302   0.6878   0.2177   0.5867     7680
       whit_a_0p20   0.1456   0.1131   0.0222   0.7196   0.1907   0.5850     7680
       whit_a_0p10   0.1478   0.1145   0.0186   0.7352   0.1778   0.5791     7680
       whit_a_0p05   0.1501 

## Randomly drop a three month gap during the year

In [17]:
dropped_months = 3
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1621   0.1277  -0.0012   0.5926   0.3137   0.6334     7680
        climo_mean   0.1239   0.0921   0.0010   0.7943   0.1601   0.7844     7680
      climo_median   0.1278   0.0888   0.0053   0.8164   0.1473   0.7732     7680
             conor   0.1095   0.0825  -0.0009   0.8370   0.1251   0.8314     7680
     interp_clim_a   0.1222   0.0954  -0.0001   0.6934   0.2369   0.8059     7680
     interp_clim_b   0.1225   0.0957   0.0001   0.6921   0.2382   0.8049     7680
     interp_clim_c   0.1219   0.0940   0.0020   0.7045   0.2305   0.8032     7680
       whit_a_0p50   0.1532   0.1176  -0.0023   0.7152   0.2179   0.6727     7680
       whit_a_0p20   0.1549   0.1185  -0.0028   0.7549   0.1867   0.6727     7680
       whit_a_0p10   0.1576   0.1204  -0.0032   0.7726   0.1726   0.6674     7680
       whit_a_0p05   0.1604   0.1226  -0.0036   0.

## Randomly drop a three month gap during the growing season (Apr-Sept)

In [18]:
dropped_months = 3
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1567   0.1230  -0.0681   0.6529   0.2498   0.6046     7680
        climo_mean   0.1226   0.0886   0.0085   0.6967   0.2862   0.7001     7680
      climo_median   0.1284   0.0857   0.0185   0.7054   0.2882   0.6777     7680
             conor   0.1063   0.0787   0.0068   0.7648   0.2221   0.7745     7680
     interp_clim_a   0.1187   0.0922  -0.0298   0.6748   0.2680   0.7414     7680
     interp_clim_b   0.1188   0.0923  -0.0300   0.6748   0.2678   0.7413     7680
     interp_clim_c   0.1184   0.0904  -0.0248   0.6791   0.2690   0.7358     7680
       whit_a_0p50   0.1505   0.1146  -0.0341   0.7259   0.2168   0.5969     7680
       whit_a_0p20   0.1521   0.1156  -0.0261   0.7580   0.1954   0.5949     7680
       whit_a_0p10   0.1542   0.1172  -0.0229   0.7731   0.1848   0.5906     7680
       whit_a_0p05   0.1562  

## Randomly drop a three month gap during the non-growing season (Oct-Mar)

In [19]:
dropped_months = 3
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a three month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a three month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1715   0.1355   0.0779   0.5978   0.3290   0.5513     7680
        climo_mean   0.1285   0.0974   0.0003   0.6845   0.1973   0.6799     7680
      climo_median   0.1314   0.0937  -0.0003   0.7087   0.1815   0.6675     7680
             conor   0.1156   0.0887  -0.0005   0.7426   0.1602   0.7408     7680
     interp_clim_a   0.1294   0.1014   0.0391   0.6411   0.2631   0.7121     7680
     interp_clim_b   0.1299   0.1019   0.0399   0.6400   0.2646   0.7106     7680
     interp_clim_c   0.1292   0.1003   0.0388   0.6532   0.2552   0.7096     7680
       whit_a_0p50   0.1596   0.1244   0.0422   0.6667   0.2502   0.5604     7680
       whit_a_0p20   0.1618   0.1253   0.0343   0.7022   0.2202   0.5543     7680
       whit_a_0p10   0.1653   0.1277   0.0311   0.7195   0.2062   0.5447     7680
       whit_a_0p05   0.168

## Randomly drop a four month gap during the year

In [20]:
dropped_months = 4
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1911   0.1522   0.0018   0.4942   0.3925   0.4935     7680
        climo_mean   0.1252   0.0938   0.0030   0.7878   0.1669   0.7825     7680
      climo_median   0.1300   0.0910   0.0075   0.8102   0.1541   0.7684     7680
             conor   0.1133   0.0863   0.0021   0.8209   0.1405   0.8220     7680
     interp_clim_a   0.1339   0.1062   0.0024   0.6410   0.2797   0.7740     7680
     interp_clim_b   0.1345   0.1066   0.0028   0.6388   0.2819   0.7719     7680
     interp_clim_c   0.1335   0.1047   0.0047   0.6522   0.2733   0.7716     7680
       whit_a_0p50   0.1737   0.1339   0.0007   0.6492   0.2716   0.5876     7680
       whit_a_0p20   0.1761   0.1352  -0.0000   0.6938   0.2365   0.5886     7680
       whit_a_0p10   0.1799   0.1382  -0.0005   0.7132   0.2210   0.5810     7680
       whit_a_0p05   0.1839   0.1413  -0.0010   0.7

## Randomly drop a four month gap during the growing season (Apr-Sept)

In [21]:
dropped_months = 4
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1847   0.1482  -0.0807   0.5645   0.3051   0.5337     7680
        climo_mean   0.1203   0.0889   0.0050   0.7561   0.2211   0.7550     7680
      climo_median   0.1245   0.0859   0.0137   0.7755   0.2126   0.7417     7680
             conor   0.1087   0.0813   0.0041   0.7973   0.1836   0.7998     7680
     interp_clim_a   0.1284   0.1024  -0.0379   0.6603   0.2631   0.7569     7680
     interp_clim_b   0.1286   0.1025  -0.0380   0.6597   0.2634   0.7561     7680
     interp_clim_c   0.1269   0.1002  -0.0335   0.6700   0.2588   0.7553     7680
       whit_a_0p50   0.1674   0.1282  -0.0422   0.6856   0.2363   0.5758     7680
       whit_a_0p20   0.1695   0.1290  -0.0332   0.7201   0.2147   0.5705     7680
       whit_a_0p10   0.1729   0.1315  -0.0298   0.7349   0.2050   0.5613     7680
       whit_a_0p05   0.1765   

## Randomly drop a four month gap during the non-growing season (Oct-Mar)

In [22]:
dropped_months = 4
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a four month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a four month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.1971   0.1566   0.0847   0.5230   0.3968   0.4724     7680
        climo_mean   0.1262   0.0955   0.0003   0.7312   0.1762   0.7322     7680
      climo_median   0.1297   0.0924   0.0009   0.7543   0.1617   0.7190     7680
             conor   0.1158   0.0883  -0.0005   0.7728   0.1481   0.7745     7680
     interp_clim_a   0.1377   0.1084   0.0425   0.6271   0.2865   0.7245     7680
     interp_clim_b   0.1386   0.1091   0.0434   0.6248   0.2889   0.7218     7680
     interp_clim_c   0.1376   0.1075   0.0428   0.6387   0.2792   0.7221     7680
       whit_a_0p50   0.1782   0.1386   0.0441   0.6309   0.2856   0.5213     7680
       whit_a_0p20   0.1797   0.1389   0.0341   0.6679   0.2514   0.5192     7680
       whit_a_0p10   0.1834   0.1415   0.0299   0.6851   0.2360   0.5098     7680
       whit_a_0p05   0.1874

## Randomly drop a six month gap during the year

In [23]:
dropped_months = 6
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the year',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the year
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2500   0.2018  -0.0001   0.3233   0.5231   0.2046     7680
        climo_mean   0.1237   0.0917   0.0015   0.7954   0.1597   0.7885     7680
      climo_median   0.1281   0.0886   0.0060   0.8182   0.1466   0.7760     7680
             conor   0.1161   0.0871  -0.0001   0.8146   0.1433   0.8137     7680
     interp_clim_a   0.1570   0.1262   0.0007   0.5593   0.3414   0.6810     7680
     interp_clim_b   0.1580   0.1270   0.0012   0.5562   0.3444   0.6760     7680
     interp_clim_c   0.1557   0.1248   0.0029   0.5708   0.3348   0.6833     7680
       whit_a_0p50   0.2245   0.1757  -0.0008   0.5019   0.3843   0.3596     7680
       whit_a_0p20   0.2231   0.1728  -0.0014   0.5586   0.3399   0.3875     7680
       whit_a_0p10   0.2256   0.1740  -0.0019   0.5844   0.3195   0.3915     7680
       whit_a_0p05   0.2290   0.1764  -0.0024   0.59

## Randomly drop a six month gap during the growing season (Apr-Sept)

In [24]:
dropped_months = 6
months = [4, 5, 6, 7, 8, 9]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue
            
    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the growing season (Apr-Sept)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the growing season (Apr-Sept)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2481   0.2004  -0.0662   0.3521   0.4664   0.2335     7680
        climo_mean   0.1237   0.0922   0.0048   0.7882   0.1789   0.7791     7680
      climo_median   0.1279   0.0889   0.0109   0.8114   0.1659   0.7677     7680
             conor   0.1164   0.0875   0.0045   0.8096   0.1611   0.8042     7680
     interp_clim_a   0.1557   0.1252  -0.0307   0.5702   0.3226   0.6814     7680
     interp_clim_b   0.1562   0.1256  -0.0305   0.5680   0.3246   0.6788     7680
     interp_clim_c   0.1539   0.1232  -0.0276   0.5817   0.3162   0.6836     7680
       whit_a_0p50   0.2222   0.1730  -0.0461   0.5225   0.3464   0.3748     7680
       whit_a_0p20   0.2209   0.1698  -0.0378   0.5754   0.3113   0.3961     7680
       whit_a_0p10   0.2239   0.1712  -0.0337   0.5978   0.2969   0.3952     7680
       whit_a_0p05   0.2278   0

## Randomly drop a six month gap during the non-growing season (Oct-Mar)

In [25]:
dropped_months = 6
months = [10, 11, 12, 1, 2, 3]

output_list = []
for point_id, year, window_df, year_month_mask in generate_windows(point_id_list, months=months):        

    # For the target year, identify all the gap windows with at least 1 month of data
    # This approach is assuming the gap will be 2 or months in the gap
    gap_mask = window_df['etf'].notna()
    for i in range(dropped_months-1):
        gap_mask = gap_mask | window_df['etf'].notna().shift(-(i+1))
        
    tgt_mask = year_month_mask & gap_mask
    if not tgt_mask.any():
        print(f'{point_id} - {year} - no unmasked months, skipping')
        continue

    tgt_indices = window_df.loc[tgt_mask].sample(n=1).index

    # Add an extra month index for each dropped month
    tgt_indices.freq = 'ms'
    for i in range(dropped_months-1):
        tgt_indices = tgt_indices.append(pd.DatetimeIndex([tgt_indices[-1] + pd.DateOffset(months=1)]))
    
    values = compute_filled_values(window_df, tgt_indices, point_id)

    # Only keep values dictionaries that had data originally
    values = [v for v in values if not np.isnan(v['original'])]

    # Only keep 1 of the filled values from the window
    output_list.extend(random.sample(values, 1))

    del window_df, tgt_mask, tgt_indices, gap_mask
    
output_df = pd.DataFrame(output_list)
comparison_stats(
    output_df, x_col='original', y_cols=comparison_cols, 
    title='Randomly drop a six month gap during the non-growing season (Oct-Mar)',
    print_flag=True, write_flag=True
)

Randomly drop a six month gap during the non-growing season (Oct-Mar)
            method     rmse      mae      mbe        m        b       r2        n
       interpolate   0.2529   0.2032   0.0738   0.3548   0.5395   0.2364     7680
        climo_mean   0.1253   0.0937   0.0023   0.7875   0.1557   0.7792     7680
      climo_median   0.1296   0.0905   0.0053   0.8115   0.1413   0.7667     7680
             conor   0.1171   0.0882   0.0007   0.8128   0.1359   0.8073     7680
     interp_clim_a   0.1583   0.1263   0.0380   0.5712   0.3476   0.6874     7680
     interp_clim_b   0.1600   0.1275   0.0392   0.5669   0.3519   0.6806     7680
     interp_clim_c   0.1579   0.1252   0.0395   0.5832   0.3404   0.6871     7680
       whit_a_0p50   0.2303   0.1806   0.0539   0.5153   0.4038   0.3609     7680
       whit_a_0p20   0.2297   0.1785   0.0454   0.5630   0.3609   0.3779     7680
       whit_a_0p10   0.2321   0.1797   0.0405   0.5838   0.3410   0.3777     7680
       whit_a_0p05   0.2353 

### Old plotting function code

In [26]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.2}, 'line_kws': {'color': 'red'}},
# )

In [27]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='hist')

In [28]:
# sns.pairplot(output_df[['original'] + comparison_cols], corner=True, kind='kde')

In [29]:
# g = sns.pairplot(
#     output_df, x_vars = ['original'], y_vars = comparison_cols, kind='reg', 
#     plot_kws={'scatter_kws': {'s': 1, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )
# for ax in g.axes.flatten():
#     if ax:
#         ax.set_xlim(0, 1.2)
#         ax.set_ylim(0, 1.2)
#         ax.set_aspect('equal')
#         ax.set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2])
#         ax.axline((0, 0), slope=1, color='gray', linestyle='--')  # linewidth=1
# plt.show()

In [30]:
# sns.pairplot(
#     output_df[['original'] + comparison_cols], corner=True, kind='reg',
#     plot_kws={'scatter_kws': {'s': 2, 'alpha': 0.1}, 'line_kws': {'color': 'red'}},
# )